# 监视数据偏移

随时间推移，由于特征数据的变化趋势，模型在准确预测方面的有效性可能会降低。此现象称为数据偏移，请务必监视机器学习解决方案以检测数据偏移，以便在必要时重新训练模型。

在本实验室中，你将为数据集配置数据偏移监视。

## 准备工作

除了最新版本的 **azureml-sdk** 和 **azureml-widgets** 包之外，还需要 **azureml-datadrift** 包才能在此笔记本中运行代码。运行下面的单元以验证是否已安装它。

In [ ]:
!pip show azureml-datadrift

## 连接到工作区

安装必需的 SDK 包后，就可以连接到工作区了。

> **备注**：如果尚未与 Azure 订阅建立经过身份验证的会话，则系统将提示你通过执行以下操作进行身份验证：单击链接，输入验证码，然后登录到 Azure。

In [ ]:
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to work with', ws.name)

## 创建基线数据集

要监视数据集是否存在数据偏移，必须注册一个基线数据集（通常是用于训练模型的数据集），以便与将来收集的数据进行比较。 

In [ ]:
from azureml.core import Datastore, Dataset


# Upload the baseline data
default_ds = ws.get_default_datastore()
default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'],
                       target_path='diabetes-baseline',
                       overwrite=True, 
                       show_progress=True)

# Create and register the baseline dataset
print('Registering baseline dataset...')
baseline_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-baseline/*.csv'))
baseline_data_set = baseline_data_set.register(workspace=ws, 
                           name='diabetes baseline',
                           description='diabetes baseline data',
                           tags = {'format':'CSV'},
                           create_new_version=True)

print('Baseline dataset registered!')

## 创建目标数据集

随时间推移，可以收集与基线训练数据具有相同特征的新数据。要将新数据与基线数据进行比较，必须定义目标数据集，其中包括要用于分析数据偏移的特征以及表示新数据为最新状态的时间点的时间戳字段 - 这样你就能够测量时间间隔内的数据偏移。时间戳可以是数据集本身的字段，也可以是从用于存储数据的文件夹和文件名模式中派生的。例如，可以将新数据存储在文件夹层次结构中，其中依次包含表示年份的文件夹、表示月份的文件夹和表示某日的文件夹；或者你可以仅对如下文件名中的年、月、日进行编码：*data_2020-01-29.csv*；这是在以下代码中采用的方法：

In [ ]:
import datetime as dt
import pandas as pd

print('Generating simulated data...')

# Load the smaller of the two data files
data = pd.read_csv('data/diabetes2.csv')

# We'll generate data for the past 6 weeks
weeknos = reversed(range(6))

file_paths = []
for weekno in weeknos:
    
    # Get the date X weeks ago
    data_date = dt.date.today() - dt.timedelta(weeks=weekno)
    
    # Modify data to ceate some drift
    data['Pregnancies'] = data['Pregnancies'] + 1
    data['Age'] = round(data['Age'] * 1.2).astype(int)
    data['BMI'] = data['BMI'] * 1.1
    
    # Save the file with the date encoded in the filename
    file_path = 'data/diabetes_{}.csv'.format(data_date.strftime("%Y-%m-%d"))
    data.to_csv(file_path)
    file_paths.append(file_path)

# Upload the files
path_on_datastore = 'diabetes-target'
default_ds.upload_files(files=file_paths,
                       target_path=path_on_datastore,
                       overwrite=True,
                       show_progress=True)

# Use the folder partition format to define a dataset with a 'date' timestamp column
partition_format = path_on_datastore + '/diabetes_{date:yyyy-MM-dd}.csv'
target_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, path_on_datastore + '/*.csv'),
                                                       partition_format=partition_format)

# Register the target dataset
print('Registering target dataset...')
target_data_set = target_data_set.with_timestamp_columns('date').register(workspace=ws,
                                                                          name='diabetes target',
                                                                          description='diabetes target data',
                                                                          tags = {'format':'CSV'},
                                                                          create_new_version=True)

print('Target dataset registered!')

## 创建数据偏移监视器

现在可为糖尿病数据创建数据偏移监视器。数据偏移监视器将定期运行或按需运行，以便比较基线数据集与目标数据集（随时间推移，目标数据集中会添加新数据）。

### 创建计算目标

要运行数据偏移监视器，你需要一个计算目标。运行以下单元格以指定计算群集（如果不存在，将创建群集）。

> **重要提示**：在运行以下代码之前，请先将代码中的 *your-compute-cluster* 更改为你的计算群集的名称！群集名称必须是长度在 2 到 16 个字符之间的全局唯一名称。有效字符是字母、数字和 - 字符。

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "your-compute-cluster"

try:
    # Check for existing compute target
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)
    

> **备注**：计算实例和群集是基于标准的 Azure 虚拟机映像。对于本练习，建议使用 *Standard_DS11_v2* 映像来实现成本和性能的最佳平衡。如果你的订阅配额不包含此映像，请选择其他映像；但请记住，较大的映像可能会产生更高的成本，但较小的映像可能不够完成任务。或者，可以请 Azure 管理员扩展你的配额。

### 定义数据偏移监视器

现在你可以使用“**DataDriftDetector**”类来定义数据的数据偏移监视器。可以指定要针对数据偏移监视的特征、用于运行监视过程的计算目标的名称、比较数据的频率、应触发警报的数据偏移阈值以及允许数据收集的延迟（以小时为单位）。

In [ ]:
from azureml.datadrift import DataDriftDetector

# set up feature list
features = ['Pregnancies', 'Age', 'BMI']

# set up data drift detector
monitor = DataDriftDetector.create_from_datasets(ws, 'mslearn-diabates-drift', baseline_data_set, target_data_set,
                                                      compute_target=cluster_name, 
                                                      frequency='Week', 
                                                      feature_list=features, 
                                                      drift_threshold=.3, 
                                                      latency=24)
monitor

## 回填数据偏移监视器

你有一个基线数据集和一个目标数据集，目标数据集包括为期六周的每周模拟数据收集。你可以用此来回填监视器，使监视器可以分析原始基线和目标数据之间的数据偏移。

> **注意** 这可能需要一些时间才能运行，因为必须启动计算目标才能运行回填分析。小组件可能不会始终更新以显示状态，因此请单击此链接以在 Azure 机器学习工作室中查看试验状态！

In [ ]:
from azureml.widgets import RunDetails

backfill = monitor.backfill(dt.datetime.now() - dt.timedelta(weeks=6), dt.datetime.now())

RunDetails(backfill).show()
backfill.wait_for_completion()

## 分析数据偏移

可以使用以下代码来检查在回填运行中收集的时间点的数据偏移。

In [ ]:
drift_metrics = backfill.get_metrics()
for metric in drift_metrics:
    print(metric, drift_metrics[metric])

还可以通过以下步骤在 [Azure 机器学习工作室](https://ml.azure.com)中可视化数据偏移指标：

1. 在“**数据集**”页上，查看“**数据集监视器**”选项卡。
2. 单击要查看的数据偏移监视器。
3. 选择要查看数据偏移指标的日期范围（如果柱状图未显示多周的数据，请等待一分钟左右再单击“**刷新**”）。
4. 检查“**偏移概述**”部分顶部的图表，其中显示总体偏移量以及每个特征的偏移贡献。
5. 浏览“**特征详细信息**”部分底部的图表，可在此查看各个特征的各种偏移度量值。

> **备注**：要了解数据偏移指标，请参阅 Azure 机器学习文档中的[如何监视数据集](https://docs.microsoft.com/azure/machine-learning/how-to-monitor-datasets#understanding-data-drift-results)。

## 进一步探索

本实验室旨在介绍数据偏移监视的概念和原则。要详细了解如何使用数据集监视数据偏移，请参阅 Azure 机器学习文档中的[检测数据集上的数据偏移](https://docs.microsoft.com/azure/machine-learning/how-to-monitor-datasets)。

还可以从已发布的服务收集数据，并将其用作监视数据偏移的目标数据集。有关详细信息，请参阅[从生产环境中的模型收集数据](https://docs.microsoft.com/azure/machine-learning/how-to-enable-data-collection)。
